In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install gymnasium
!pip install pygame
!pip install wheel setuptools
!pip install swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379367 sha256=02a8c98464434f527806005df839f6072c62b5b45505f25cc8776a42132aec23
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


In [2]:
!pip uninstall -y box2d-py
!pip install box2d pygame swig
!pip install "gymnasium[box2d]" --no-deps

Found existing installation: box2d-py 2.3.5
Uninstalling box2d-py-2.3.5:
  Successfully uninstalled box2d-py-2.3.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 58.8 MB/s eta 0:00:00


In [101]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class ActorCriticPPO(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dims=(64, 64)):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU()
        )
        self.actor_layer = nn.Linear(hidden_dims[1], output_dim)
        self.critic_layer = nn.Linear(hidden_dims[1], 1)

    def forward(self, x):
        x = self.net(x)
        action_probs = F.softmax(self.actor_layer(x), dim=-1)
        value = self.critic_layer(x)
        return action_probs, value


In [105]:
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
import numpy as np
import copy

class PPOAgent:
    def __init__(self, env_id, num_episodes=1000, max_steps=500, epsilon=float('inf'), gamma=0.99, lambda_GAE=1, lr=1e-3, num_steps=0, num_envs=8, vectorization_mode = "sync", seed=123):
        # using vectorized environments to boost training speed
        self.env = gym.make_vec(env_id, num_envs=num_envs, vectorization_mode=vectorization_mode)
        self.num_envs = num_envs
        self.num_episodes = num_episodes
        self.max_steps = max_steps
        self.epsilon = epsilon
        self.gamma = gamma
        self.lambda_GAE = lambda_GAE
        self.lr = lr
        self.num_steps = num_steps
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = ActorCriticPPO(self.env.single_observation_space.shape[0], self.env.single_action_space.n).to(self.device)
        self.old_policy_net = copy.deepcopy(self.policy_net)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        # added scheduler after observing divergence after getting close to solving
        self.scheduler = StepLR(self.optimizer, step_size=100, gamma=0.9)
        self.loss = nn.MSELoss()
        self.seed = seed

    # choosing action from policy's probability distribution
    def choose_action(self, state):
        probs, _ = self.old_policy_net(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action

    # computing GAE
    def compute_returns(self, rewards, values, next_value):
        rewards = torch.stack(rewards)
        values = torch.cat([values, next_value.unsqueeze(0)], dim=0)

        T, N = rewards.shape
        advantages = torch.zeros_like(rewards)
        gae = torch.zeros(N, device=rewards.device)
        for t in reversed(range(T)):
            # temporal difference error
            td = rewards[t] + self.gamma * values[t + 1] - values[t]
            # higher labmda -> more sampling, lower lambda -> more bootstrapping
            gae = td + self.gamma * self.lambda_GAE * gae
            advantages[t] = gae

        # compute returns by adding value to advantage
        returns = advantages + values[:-1]
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)

        # normalize advantage across timesteps and environments
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        return returns, advantages

    # computing the gamma decaying rewards
    # def compute_returns(self, rewards, values, next_value):
    #     """
    #     Args:
    #         rewards: torch.Tensor of shape [T, N] where
    #                 T = rollout steps, N = num_envs
    #     Returns:
    #         returns: torch.Tensor of shape [T, N], normalized
    #     """
    #     rewards = torch.stack(rewards)

    #     T, N = rewards.shape
    #     returns = torch.zeros_like(rewards)
    #     R = torch.zeros(N, device=rewards.device)
    #     for t in reversed(range(T)):
    #         R = rewards[t] + self.gamma * R
    #         returns[t] = R

    #     # Normalize returns across all timesteps and environments
    #     returns = (returns - returns.mean()) / (returns.std() + 1e-8)
    #     advantages = returns - values

    #     return returns, advantages


    def train(self):
        episode_rewards = []
        episode_steps = []

        for episode in tqdm(range(self.num_episodes)):
            state, _ = self.env.reset(seed=self.seed)
            done = np.zeros(self.num_envs, dtype=bool)
            episode_reward = np.zeros(self.num_envs)
            values, rewards, log_probs, old_log_probs, old_values = [], [], [], [], []
            done_mask = np.zeros(self.num_envs, dtype=bool)
            done_steps = np.zeros(self.num_envs)
            steps = 0

            while not np.all(done_mask) and steps < self.max_steps:
                steps += 1
                state_tensor = torch.tensor(state, dtype=torch.float32).to(self.device)
                action_probs, value = self.policy_net(state_tensor)
                action_dist = torch.distributions.Categorical(action_probs)
                action = action_dist.sample()
                log_prob = action_dist.log_prob(action)

                with torch.no_grad():
                    old_action_probs, old_value = self.old_policy_net(state_tensor)
                    old_action_dist = torch.distributions.Categorical(old_action_probs)
                    old_log_prob = old_action_dist.log_prob(action)

                next_state, reward, terminated, truncated, _ = self.env.step(action.cpu().numpy())
                done = np.logical_or(terminated, truncated)
                done_steps = np.where(np.logical_and(done, ~done_mask), steps, done_steps)
                done_mask = np.logical_or(done_mask, done)
                # record when each environment is done
                reward = np.where(done_mask, 0.0, reward)

                # saves the values, rewards, log_probs which are used to calculate the n_step returns, actor loss, and critic loss
                values.append(value.squeeze())
                rewards.append(torch.tensor(reward, dtype=torch.float32).to(self.device))  # shape: (num_envs,)
                log_probs.append(log_prob)
                old_log_probs.append(old_log_prob)
                old_values.append(old_value.squeeze())

                episode_reward += reward
                state = next_state


                # finish full trajectory, then update
                if self.num_steps == 0:
                    if np.any(done):
                        with torch.no_grad():
                            next_state_tensor = torch.tensor(next_state, dtype=torch.float32).to(self.device)
                            _, next_value = self.policy_net(next_state_tensor)
                            done_tensor = torch.tensor(done, dtype=torch.float32).to(self.device)
                            next_value = next_value.squeeze() * (1 - done_tensor)

                        values = torch.stack(values)  # shape: (n_steps, num_envs)
                        returns, advantages = self.compute_returns(rewards, values, next_value)  # shape: (n_steps, num_envs)
                        returns, values, advantages = returns.transpose(0, 1), values.transpose(0, 1), advantages.transpose(0, 1)  # shape: (num_envs, n_steps)
                        log_probs, old_log_probs = torch.stack(log_probs).transpose(0, 1), torch.stack(old_log_probs).transpose(0, 1)  # shape: (num_envs, n_steps)
                        prob_ratio = torch.exp(log_probs - old_log_probs)

                        surrogate = torch.min(prob_ratio * advantages, prob_ratio.clamp(1 - self.epsilon, 1 + self.epsilon) * advantages)
                        actor_loss = -surrogate.mean()
                        critic_loss = self.loss(returns, values)

                        loss = actor_loss + critic_loss
                        self.optimizer.zero_grad()
                        loss.backward()
                        self.optimizer.step()
                        # self.scheduler.step()

                        values = []
                        rewards = []
                        log_probs = []
                        old_log_probs = []

                # every n steps for each environment, calculate losses, update the actor & critic, then refresh the saved lists
                else:
                    if (steps % self.num_steps == 0) or np.any(done):
                        with torch.no_grad():
                            next_state_tensor = torch.tensor(next_state, dtype=torch.float32).to(self.device)
                            _, next_value = self.policy_net(next_state_tensor)
                            done_tensor = torch.tensor(done, dtype=torch.float32).to(self.device)
                            next_value = next_value.squeeze() * (1 - done_tensor)

                        values = torch.stack(values)  # shape: (n_steps, num_envs)
                        old_values = torch.stack(old_values).transpose(0, 1)
                        returns, advantages = self.compute_returns(rewards, values, next_value)  # shape: (n_steps, num_envs)
                        returns, values, advantages = returns.transpose(0, 1), values.transpose(0, 1), advantages.transpose(0, 1)  # shape: (num_envs, n_steps)
                        log_probs, old_log_probs = torch.stack(log_probs).transpose(0, 1), torch.stack(old_log_probs).transpose(0, 1)  # shape: (num_envs, n_steps)
                        prob_ratio = torch.exp(log_probs - old_log_probs.detach())

                        clipped = (prob_ratio > 1 + self.epsilon) | (prob_ratio < 1 - self.epsilon)
                        clip_fraction = clipped.float().mean().item()
                        # print(f"Clipped fraction: {clip_fraction:.2%}")

                        surrogate = torch.min(prob_ratio * advantages, prob_ratio.clamp(1 - self.epsilon, 1 + self.epsilon) * advantages)
                        actor_loss = -surrogate.mean()

                        # clipped_values = torch.clamp(values, old_values.detach() * (1 - self.epsilon), old_values.detach() * (1 + self.epsilon))
                        # critic_loss = min(self.loss(returns, values), self.loss(returns, clipped_values))
                        critic_loss = self.loss(returns, values)

                        loss = actor_loss + 0.5 * critic_loss
                        self.optimizer.zero_grad()
                        loss.backward()
                        self.optimizer.step()
                        # self.scheduler.step()
                        # if episode % 10 == 0:
                        #     print("VALUES", values, "\n", "RETURNS", returns)

                        values, rewards, log_probs, old_log_probs, old_values = [], [], [], [], []


            self.old_policy_net = copy.deepcopy(self.policy_net)

            episode_rewards.append(episode_reward)
            episode_steps.append(steps)

            if episode % 20 == 0:
               print('Episode {}\tlengths: {}\treward: {}]\tfull length: {}'.format(episode, done_steps, episode_reward, steps))
            if episode % 10 == 0:
                print(f"\n[Episode {episode}]")
                print(f"Reward (mean): {np.mean(episode_reward):.2f}")
                print(f"Actor Loss: {actor_loss.item():.4f} | Critic Loss: {critic_loss.item():.4f}")
                print(f"Prob Ratio - mean: {prob_ratio.mean().item():.4f}, max: {prob_ratio.max().item():.4f}, min: {prob_ratio.min().item():.4f}")

            episode_rewards.append(episode_reward)
            episode_steps.append(steps)

        self.env.close()
        return np.array(episode_rewards), np.array(episode_steps)


In [ ]:
import gymnasium as gym

env_id = 'LunarLander-v3'
num_episodes = 500
max_steps = 500
lr = 1e-4


ppo_model =  PPOAgent(env_id, num_episodes=num_episodes, max_steps=max_steps, lr=lr, epsilon=0.2, num_envs=8, num_steps=8)

rewards, steps = ppo_model.train()



  0%|          | 1/500 [00:00<04:14,  1.96it/s]

Episode 0	lengths: [ 64.  69. 109.  97.  89. 100.  90.  72.]	reward: [-167.85368813 -112.30575797  -19.21825992 -111.71412477 -243.36392964
  -20.24118961  -58.12143199  -61.3414641 ]]	full length: 109

[Episode 0]
Reward (mean): -99.27
Actor Loss: -0.0002 | Critic Loss: 0.9986
Prob Ratio - mean: 1.0005, max: 1.0094, min: 0.9910


  2%|▏         | 11/500 [00:06<05:23,  1.51it/s]


[Episode 10]
Reward (mean): -31.01
Actor Loss: -0.0012 | Critic Loss: 1.0522
Prob Ratio - mean: 0.9998, max: 1.0099, min: 0.9934


  4%|▍         | 21/500 [00:11<04:00,  1.99it/s]

...

 94%|█████████▍| 471/500 [13:06<01:01,  2.12s/it]


[Episode 470]
Reward (mean): 29.12
Actor Loss: 0.0120 | Critic Loss: 1.0777
Prob Ratio - mean: 1.0043, max: 1.1285, min: 0.7898


 96%|█████████▌| 481/500 [13:28<00:40,  2.12s/it]

Episode 480	lengths: [381.   0. 278. 233. 293.   0. 248. 382.]	reward: [  86.83824918  134.00459355   86.16381746   86.95362427   73.09975959
  126.24677061  151.64064089 -108.56633362]]	full length: 500

[Episode 480]
Reward (mean): 79.55
Actor Loss: 0.0030 | Critic Loss: 0.7645
Prob Ratio - mean: 1.0016, max: 1.0333, min: 0.9782


 97%|█████████▋| 483/500 [13:33<00:38,  2.29s/it]

In [ ]:
import gymnasium as gym
import torch
import numpy as np
from gymnasium.wrappers import RecordVideo
import os

# Create folder to save the video
video_folder = "./video"
os.makedirs(video_folder, exist_ok=True)

# Wrap the environment with RecordVideo
env = gym.make('LunarLander-v3', render_mode='rgb_array')
env = RecordVideo(env, video_folder=video_folder, episode_trigger=lambda e: True)

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

episode_rewards = []

for i in range(num_episodes):
    state, _ = env.reset()
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
    episode_reward = 0
    done = False

    while not done:
        with torch.no_grad():
            action_probs, _ = ppo_model.policy_net(state)
            # action_dist = torch.distributions.Categorical(action_probs)
            # action = action_dist.sample().item()
            action = torch.argmax(action_probs, dim=1).item()

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        episode_reward += reward

        state = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0).to(device)

    episode_rewards.append(episode_reward)
    print(f"Episode {i+1} Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")


In [64]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class LinearValueEnv(gym.Env):
    def __init__(self, gamma=0.99, episode_length=100):
        super().__init__()
        self.gamma = gamma
        self.episode_length = episode_length
        self.current_step = 0

        # Observation: continuous scalar in [-1, 1]
        self.observation_space = spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

        # Action: continuous scalar (not used)
        self.action_space = spaces.Discrete(1)

        self.state = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.random.uniform(-1.0, 1.0, size=(1,)).astype(np.float32)
        self.current_step = 0
        return self.state.copy(), {}

    def step(self, action):
        # Reward is simply the state value
        reward = float(self.state[0])
        self.current_step += 1

        terminated = self.current_step >= self.episode_length
        truncated = False
        return self.state.copy(), reward, terminated, truncated, {}

    def render(self):
        print(f"State: {self.state}")

    def close(self):
        pass




In [65]:
from gymnasium.envs.registration import register

register(
    id="LinearValue-v0",
    entry_point="__main__:LinearValueEnv",  # if you're running in a script
    max_episode_steps=100
)


In [100]:
env_id = "LinearValue-v0"
num_episodes = 1000
max_steps = 500
lr = 1e-4


ppo_model_value =  PPOAgent(env_id, num_episodes=num_episodes, max_steps=max_steps, lr=lr, epsilon=0.2, num_envs=2, num_steps=0)

rewards, steps = ppo_model_value.train()


  0%|          | 1/1000 [00:00<12:09,  1.37it/s]

Episode 0	lengths: [100. 100.]	reward: [-72.96960783  84.58938628]]	full length: 100

[Episode 0]
Reward (mean): 5.81
Actor Loss: 0.3642 | Critic Loss: 0.9067
Prob Ratio - mean: 1.0000, max: 1.0000, min: 1.0000


  1%|          | 11/1000 [00:03<04:25,  3.73it/s]


[Episode 10]
Reward (mean): -41.61
Actor Loss: 0.2812 | Critic Loss: 1.0306
Prob Ratio - mean: 1.0000, max: 1.0000, min: 1.0000


  2%|▏         | 21/1000 [00:06<04:13,  3.86it/s]

...

Episode 200	lengths: [100. 100.]	reward: [-81.71754402 -39.18708792]]	full length: 100

[Episode 200]
Reward (mean): -60.45
Actor Loss: 0.2027 | Critic Loss: 0.9949
Prob Ratio - mean: 1.0000, max: 1.0000, min: 1.0000


 21%|██        | 211/1000 [00:57<03:23,  3.87it/s]


[Episode 210]
Reward (mean): -59.89
Actor Loss: 0.2012 | Critic Loss: 1.0341
Prob Ratio - mean: 1.0000, max: 1.0000, min: 1.0000


 21%|██▏       | 213/1000 [00:58<03:35,  3.65it/s]


KeyboardInterrupt: 